In [ ]:
import os
import shutil
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def filter_macro_indicators(df):
    # Define comprehensive keywords for global macroeconomic indicators
    keywords = [
        # Inflasi (CPI & Core PCE)
        'CPI', 'Consumer Price Index', 'Core CPI', 'Core PCE', 'PCE Price Index',

        # Tenaga Kerja
        'Nonfarm Payrolls', 'Non-Farm Payrolls', 'NFP', 'Employment Change',
        'Unemployment Rate', 'Jobless Claims', 'Initial Jobless Claims',

        # Suku Bunga dan Proyeksi
        'Interest Rate Decision', 'Fed Interest Rate Decision', 'Federal Funds Rate',
        'FOMC Statement', 'Dot Plot', 'Rate Statement', 'Rate Projection',
        'Monetary Policy Statement',

        # Khusus China – suku bunga dan kebijakan moneter
        '1-Year Loan Prime Rate', '5-Year Loan Prime Rate', 'Loan Prime Rate',
        'PBoC Interest Rate', 'PBoC Liquidity Injection', 'MLF Rate', 'MLO Rate',

        # Pertumbuhan Ekonomi
        'GDP', 'Gross Domestic Product',

        # PMI
        'PMI', 'Purchasing Managers', 'Manufacturing PMI', 'Services PMI',
        'Composite PMI', 'Caixin Manufacturing PMI', 'Caixin Services PMI'
    ]

    # Create a filter mask (case-insensitive search)
    mask = df['Event'].str.contains('|'.join(keywords), case=False, regex=True)

    # Apply the filter
    filtered_df = df[mask].copy()

    # Convert date to datetime if it's not already
    filtered_df['Tanggal'] = pd.to_datetime(filtered_df['Tanggal'])

    # Sort by date and currency
    filtered_df = filtered_df.sort_values(['Tanggal', 'Mata Uang'])

    return filtered_df

# Read the CSV file
df = pd.read_csv('/content/drive/My Drive/Skripsi/Bahan/DF-Makroekonomi2.csv')

# Apply the filter
filtered_data = filter_macro_indicators(df)

# Display the results grouped by currency
print("\nFiltered Macroeconomic Indicators by Currency:")
for currency in sorted(filtered_data['Mata Uang'].unique()):
    print(f"\n=== {currency} ===")
    currency_data = filtered_data[filtered_data['Mata Uang'] == currency]
    print(currency_data.to_string(index=False))

# Optional: Save filtered data to new CSV
filtered_data.to_csv('DF-Makroekonomi-filtered2.csv', index=False)


Filtered Macroeconomic Indicators by Currency:

=== CNY ===
   Tanggal Mata Uang                            Event Aktual
2018-01-10       CNY                 CPI (YoY)  (Dec)   1.8%
2018-01-10       CNY                 CPI (MoM)  (Dec)   0.3%
2018-01-18       CNY                  GDP (QoQ)  (Q4)   1.6%
2018-01-18       CNY                  GDP (YoY)  (Q4)   6.8%
2018-01-31       CNY         Manufacturing PMI  (Jan)   51.3
2018-01-31       CNY     Non-Manufacturing PMI  (Jan)   55.3
2018-02-09       CNY                 CPI (YoY)  (Jan)   1.5%
2018-02-09       CNY                 CPI (MoM)  (Jan)   0.6%
2018-02-28       CNY         Manufacturing PMI  (Feb)   50.3
2018-02-28       CNY     Non-Manufacturing PMI  (Feb)   54.4
2018-03-09       CNY                 CPI (YoY)  (Feb)   2.9%
2018-03-09       CNY                 CPI (MoM)  (Feb)   1.2%
2018-03-31       CNY         Manufacturing PMI  (Mar)   51.5
2018-03-31       CNY     Non-Manufacturing PMI  (Mar)   54.6
2018-04-11       CNY    

In [ ]:
source_file = '/content/DF-Makroekonomi-filtered2.csv'
destination_folder = '/content/drive/My Drive/Skripsi/Bahan/'

os.makedirs(destination_folder, exist_ok=True)

destination_file = os.path.join(destination_folder, 'DF-Makroekonomi-filtered2.csv')

shutil.copy2(source_file, destination_file)

print(f"File '{source_file}' copied to '{destination_file}'")


File '/content/DF-Makroekonomi-filtered2.csv' copied to '/content/drive/My Drive/Skripsi/Bahan/DF-Makroekonomi-filtered2.csv'


In [ ]:
# Load the filtered data
filtered_data = pd.read_csv('/content/drive/My Drive/Skripsi/Bahan/DF-Makroekonomi-filtered2.csv')

# Function to extract core event name
def extract_core_event(event_name):
    # Remove text within parentheses and extra spaces
    event_name = re.sub(r'\(.*\)', '', event_name).strip()
    return event_name

# Apply the function to the 'Event' column
filtered_data['Core Event'] = filtered_data['Event'].apply(extract_core_event)

# Group data by currency and core event, then count occurrences
currency_events = filtered_data.groupby(['Mata Uang', 'Core Event']).size().reset_index(name='Count')

# Create a dictionary to store the summary
currency_summary = {}

# Iterate through unique currencies and core events to build the summary
for currency in sorted(currency_events['Mata Uang'].unique()):
    currency_summary[currency] = []
    currency_df = currency_events[currency_events['Mata Uang'] == currency]
    for index, row in currency_df.iterrows():
        currency_summary[currency].append(row['Core Event'])

# Display the summary with only core event names
for currency, events in currency_summary.items():
    print(f"{currency}: {', '.join(events)}")

CNY: CPI, China Loan Prime Rate 5Y, Chinese Composite PMI, Chinese GDP YTD, Chinese Unemployment Rate, GDP, Manufacturing PMI, Non-Manufacturing PMI, PBoC Loan Prime Rate
EUR: German CPI, German GDP, German Unemployment Change, German Unemployment Rate
GBP: BoE Interest Rate Decision, CPI, CPIH, Employment Change 3M/3M, GDP, Monthly GDP 3M/3M Change, NIESR GDP Estimate, NIESR Monthly GDP Tracker, Unemployment Rate
JPY: BoJ Core CPI, BoJ Interest Rate Decision, BoJ Monetary Policy Statement, GDP, GDP Annualized, GDP Price Index, National CPI, National Core CPI, Tokyo Core CPI
USD: ADP Nonfarm Employment Change, Atlanta Fed GDPNow, CPI, Chicago PMI, Continuing Jobless Claims, Core CPI, Core PCE Price Index, Core PCE Prices, FOMC Statement, Fed Interest Rate Decision, GDP, GDP Price Index, ISM Manufacturing PMI, ISM Non-Manufacturing PMI, Initial Jobless Claims, Interest Rate Projection - 1st Yr, Interest Rate Projection - 2nd Yr, Interest Rate Projection - 3rd Yr, Interest Rate Projectio

In [ ]:
filtered_data

,Tanggal,Mata Uang,Event,Aktual,Core Event
0,2018-01-03,EUR,German Unemployment Change (Dec),-29K,German Unemployment Change
1,2018-01-03,EUR,German Unemployment Rate (Dec),5.5%,German Unemployment Rate
2,2018-01-03,USD,ISM Manufacturing PMI (Dec),59.7,ISM Manufacturing PMI
3,2018-01-04,USD,ADP Nonfarm Employment Change (Dec),250K,ADP Nonfarm Employment Change
4,2018-01-04,USD,Continuing Jobless Claims,"1,914K",Continuing Jobless Claims
...,...,...,...,...,...
5341,2024-12-27,JPY,Tokyo Core CPI (YoY) (Dec),2.4%,Tokyo Core CPI
5342,2024-12-30,USD,Chicago PMI (Dec),36.9,Chicago PMI
5343,2024-12-31,CNY,Chinese Composite PMI (Dec),52.2,Chinese Composite PMI
5344,2024-12-31,CNY,Manufacturing PMI (Dec),50.1,Manufacturing PMI
